In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import nltk
import pickle
from nltk import pos_tag
from scipy import spatial
from scipy.sparse import hstack, vstack
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from scipy import spatial
import multiprocessing
import math
import matplotlib.pyplot as plt

data_path = "/media/data/cuixuange/competition_2019/first_round/data/"
feature_path = "/media/data/cuixuange/competition_2019/first_round/feature/"
model_path = "/media/data/cuixuange/competition_2019/first_round/model/"

In [3]:
w2vmodel = pickle.load(open(model_path+"word2vec_model.pkl","rb"))
from gensim.models import KeyedVectors
save_path = model_path + "w2v_vector.kv"
w2vmodel.wv.save(save_path)
word_vectors = KeyedVectors.load(save_path, mmap='r')


In [4]:
################################################# pretrain_embedding
################################################# pretrain_embedding
################################################# pretrain_embedding
word_dim = 100
pre_emb_len = len(word_vectors.vocab.keys()) + 1
pre_emb_matrix = np.zeros((pre_emb_len,word_dim))
pre_reindex_dict = dict()
reindex = 1
for key,_ in word_vectors.vocab.items():
    pre_emb_matrix[reindex] = word_vectors[key] #填充pretrained_emb
    pre_reindex_dict[key] = reindex #key:单词值 value:embedding index
    reindex += 1

print(pre_emb_matrix.shape,type(pre_emb_matrix))
# for k,v in pre_reindex_dict.items():
#     print(k,"\t:",v)

(10148, 100) <class 'numpy.ndarray'>


In [5]:
############################################ dataset
############################################ dataset
############################################ dataset
import torch
from torch.utils.data import Dataset,DataLoader
colnames = ["query_id","query","query_title_id","title","label"]
train_data_path = data_path+"train_data.sample"
cores = multiprocessing.cpu_count()-1
sentence_limit = 50
word_dim = 100
############ valid
train_index = pickle.load(open(data_path+"train.index.pkl", "rb")) 
valid_index = pickle.load(open(data_path+"valid.index.pkl", "rb")) 

def get_seq_list(word_list):
    seq_list = np.zeros(sentence_limit)#序号0对应embedding层的零向量
    exits_word_count = 0
    for val in word_list:
        if exits_word_count > sentence_limit:
            break
        try:
            seq_list[exits_word_count] = pre_reindex_dict[val] #reindex
        except:
            continue  
        exits_word_count += 1
    return seq_list

class Dataset(torch.utils.data.Dataset):
    def __init__(self, filepath=None):
        self.data =  pd.read_csv(filepath,names=colnames,header=None,lineterminator="\n").iloc[train_index]
        self.lgb_data =  pd.read_csv(feature_path+"combine_feature.csv", header=None,lineterminator="\n").iloc[train_index]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        query = get_seq_list(str(row[1]).split(" "))    
        title = get_seq_list(str(row[3]).split(" "))
        label = row[4]
        lgb_feature = self.lgb_data.iloc[index]
        return query,title,label,lgb_feature
    
class test_Dataset(torch.utils.data.Dataset):
    def __init__(self, filepath=None):
        self.data =  pd.read_csv(filepath,names=colnames,header=None,lineterminator="\n").iloc[valid_index]
        self.lgb_data =  pd.read_csv(feature_path+"combine_feature.csv", header=None,lineterminator="\n").iloc[valid_index]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        query = get_seq_list(str(row[1]).split(" "))    
        title = get_seq_list(str(row[3]).split(" "))
        label = row[4]
        lgb_feature = self.lgb_data.iloc[index]
        return query,title,label,lgb_feature
    
train_dataset = Dataset(train_data_path)
test_dataset = test_Dataset(train_data_path)

In [6]:
############################################ dataloader collect_fn 
############################################ dataloader collect_fn 
############################################ dataloader collect_fn 
batch_size = 256

def my_collate_padding(batch_list,dim2pad=0):
    """
    query,title,label分别进行padding操作(句子长度不是等长的)
    input: list of __getitem__ data
    output: merge data to batch
    """
    list_query,list_title,list_label,list_lgb_feature = zip(*batch_list)
    batch_size = len(batch_list) # 最后一个的batchsize是不完整的,这里的batchsize需要调整
    pad_query = np.zeros((batch_size, sentence_limit))
    pad_title = np.zeros((batch_size, sentence_limit))
    for i, tmp in enumerate(list_query):
        pad_query[i] = tmp
    for i, tmp in enumerate(list_title):
        pad_title[i] = tmp
    return pad_query,pad_title,np.array(list_label),np.array(list_lgb_feature)


train_data_generator = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,collate_fn=my_collate_padding,
                                          shuffle=False, sampler=None, batch_sampler=None, 
                                          num_workers=4, pin_memory=False, drop_last=False)

test_data_generator = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,collate_fn=my_collate_padding,
                                          shuffle=False, sampler=None, batch_sampler=None, 
                                          num_workers=cores, pin_memory=False, drop_last=False)

dataloader_iterator = iter(train_data_generator)
batch_q,batch_t,batch_l,batch_lgb = dataloader_iterator.next()
print(batch_q.shape)
print(batch_t.shape)
print(batch_l.shape)
print(batch_lgb.shape)
print(batch_q[0])
print(batch_t[1])

(256, 50)
(256, 50)
(256,)
(256, 47)
[ 1.  2.  3.  1.  4.  5.  6.  7.  8.  9. 10. 11. 10. 12. 13.  7. 14. 15.
 16. 17. 18. 19. 20. 21. 22. 12. 23. 24. 25. 26. 27. 28. 29. 28.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 6.  7.  8.  9. 43. 51. 44. 12. 13.  7. 14. 15. 16. 17.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CNN_Text(nn.Module):
    
    def __init__(self):
        super(CNN_Text, self).__init__()
        
        in_channel = 1
        out_channel = 128
        class_num = 2
        dropout = 0.5
        Dim = word_dim
        kernel_list_q = [2,3,5]
        kernel_list_t = [2,3,5]
#         other_feature_len = 45
        other_feature_len = 47
        # pooling*3*（2=query+title），sim(3个pool) + lgb_feature
        last_linear_dim = out_channel*3*2 + 2*3 + other_feature_len
        
        # 0.embedding layer
        self.embed_layer = nn.Embedding(num_embeddings=pre_emb_len,embedding_dim=word_dim,sparse=False)
        self.embed_layer.weight.data.copy_(torch.from_numpy(pre_emb_matrix))
        self.embed_layer.weight.requires_grad = True #default
        
        # 1.CNN    #batch_normalize 对于训练集的效果很好
        self.cnn_parallel_query = nn.ModuleList([ nn.Sequential(
                                            nn.Conv2d(in_channel, out_channel, (K, Dim)),
                                            nn.BatchNorm2d(out_channel)
                                                            ) for K in kernel_list_q])
        self.cnn_parallel_title = nn.ModuleList([ nn.Sequential(
                                            nn.Conv2d(in_channel, out_channel, (K, Dim)),
                                            nn.BatchNorm2d(out_channel)
                                                    ) for K in kernel_list_t])
        # 2.Linear
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(last_linear_dim, class_num)

    def CNN_query(self,x):
        x = x.unsqueeze(1)  # (N, Ci, W, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.cnn_parallel_query]  # [(N, Co, W), ...]*len(Ks)
        x = [F.max_pool1d(input=i, kernel_size=i.size(2),stride=i.size(2)).squeeze(2) for i in x]
        return x
    
    def CNN_title(self,x):
        x = x.unsqueeze(1)  # (N, Ci, W, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.cnn_parallel_title]  # [(N, Co, W), ...]*len(Ks)
        x = [F.max_pool1d(input=i, kernel_size=i.size(2),stride=i.size(2)).squeeze(2) for i in x]
        return x
    
    def cosine_sim(self, query, title):
        l2_query = torch.norm(query,p=2,dim=1)
        l2_title = torch.norm(title,p=2,dim=1)
        sim_eud = torch.norm(query-title,p=2,dim=1) # 欧式距离 L2
        sim_cos = torch.sum(query * title,dim=1)/(l2_query * l2_title)
        return torch.cat((sim_cos.unsqueeze(1),sim_eud.unsqueeze(1)),dim=1)
    
    def forward(self, query,title,other_feature):
        query = self.CNN_query(self.embed_layer(query)) # [(N, Co), ...]*len(Ks)
        title = self.CNN_title(self.embed_layer(title)) # [(N, Co), ...]*len(Ks)
        sim_2dim = [self.cosine_sim(query[i],title[i]) for i in range(len(query))]
        pool_padding = torch.cat((tuple(query) + tuple(title) + tuple(sim_2dim) + tuple([other_feature])),dim=1)
        x = self.dropout(pool_padding)
        logit = self.fc1(x)  # (N, C)
        return logit

    

In [18]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight.data, mode='fan_in')
#         torch.nn.init.kaiming_uniform_(m.bias.data, mode='fan_in')
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data, mode='fan_in')
#         torch.nn.init.kaiming_uniform(m.bias.data, mode='fan_in')   #Error 偏置项为零

# net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
# net.apply(weights_init)

In [19]:
from torch.autograd import Variable
from numpy import linalg

def test_classify():
    x1 = torch.LongTensor(256,50).random_(0, 10)
    x2 = torch.LongTensor(256,50).random_(0, 10)
#     other = torch.rand(256,45)
    other = torch.rand(256,47)
    
    model = CNN_Text()
    model.apply(weights_init)
    pre1 = model(x1,x2,other)
    print(pre1.shape)
#     print(pre1[0].shape,pre1[1].shape,pre1[-1].shape)
    return pre1
    
test = test_classify()
# print(test.shape)

torch.Size([256, 2])


In [20]:
def plot(loss,mean_loss):
    from IPython import display
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plt.figure(figsize=(20,10))
    plt.clf()

    plt.title('Training...')
    plt.xlabel('iterator')
    plt.ylabel('Loss')
    plt.plot(loss)
    plt.plot(mean_loss)
    plt.show()

In [21]:
import torch
torch.cuda.set_device(3)
print(torch.cuda.current_device())

3


In [1]:
############ 训练模型
import torch.optim as optim
import matplotlib.pyplot as plt
cnn_model = CNN_Text().float().cuda()
# cnn_model.apply(weights_init)  ############ weight init
max_epochs = 2
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001,weight_decay=0.00001)
# weight = [1,1]
# class_weights = torch.FloatTensor(weight).cuda()
# lossFunc = nn.CrossEntropyLoss(weight=class_weights)
lossFunc = nn.CrossEntropyLoss()
cal_loss_list = []
cal_mean_loss_list = []

count = 0
for epoch in range(max_epochs):
    for batch_q,batch_t,batch_label,batch_lgb in train_data_generator:
        var_q = torch.from_numpy(batch_q).long().cuda()
        var_t = torch.from_numpy(batch_t).long().cuda()
        var_lgb = torch.from_numpy(batch_lgb).float().cuda()
        var_label = torch.from_numpy(batch_label).cuda()
        optimizer.zero_grad()
        outputs = cnn_model(var_q,var_t,var_lgb)
#         print(outputs)
#         break
        loss = lossFunc(outputs,var_label)
        loss.backward()
        optimizer.step()

        running_loss = loss.item()
        cal_loss_list.append(running_loss)
        count += 1
        if(count<100):
            cal_mean_loss_list.append(sum(cal_loss_list)/len(cal_loss_list))
        else:
            cal_mean_loss_list.append(sum(cal_loss_list[-100:])/100)
            
        if(count%10 == 0):
            plot(cal_loss_list,cal_mean_loss_list)
        
        if(count%200 == 0):
            torch.save(cnn_model.state_dict(),model_path+"text_CNN.model")
            print(cal_mean_loss_list[-1])

NameError: name 'CNN_Text' is not defined

In [27]:
print(cnn_model.state_dict())
torch.save(cnn_model.state_dict(),model_path+"text_CNN.model")
print('Finished Training')
pickle.dump(cnn_model,open(model_path+"text_CNN.pkl","wb"))

OrderedDict([('embed_layer.weight', tensor([[ 0.0080,  0.0038, -0.0096,  ...,  0.0109, -0.0015, -0.0016],
        [-0.4945,  0.0910, -0.3353,  ..., -0.2240, -0.1767,  0.1385],
        [-0.4775,  0.5060, -0.0590,  ..., -0.0515, -0.5452,  0.6913],
        ...,
        [-0.2672,  0.5675, -0.0059,  ...,  0.0284,  0.0258,  0.1345],
        [-0.2670,  0.2538, -0.4814,  ...,  0.3603,  0.4116,  0.1127],
        [ 0.1895, -0.0186, -0.1335,  ..., -0.0862,  0.3232,  0.4144]],
       device='cuda:3')), ('cnn_parallel_query.0.0.weight', tensor([[[[-0.0053,  0.0158, -0.0093,  ...,  0.0485, -0.0765,  0.0435],
          [-0.0502,  0.0283, -0.0337,  ..., -0.0431,  0.0477,  0.0109]]],


        [[[ 0.0332, -0.0138,  0.0627,  ...,  0.0243, -0.0041,  0.0267],
          [ 0.0346,  0.0527,  0.0172,  ...,  0.0139, -0.0706, -0.0388]]],


        [[[ 0.0528, -0.0324,  0.0845,  ..., -0.0374, -0.0381, -0.0509],
          [-0.0730,  0.0586, -0.0221,  ..., -0.0505,  0.0425,  0.0371]]],


        ...,


        [[[

In [28]:
test_model = CNN_Text().float()
state_dict = torch.load(model_path+"text_CNN.model")
test_model.load_state_dict(state_dict)
test_model = test_model.eval()

# test_model = pickle.load(open(model_path+"text_CNN.pkl","rb"))
# test_model.eval()
y_true = []
ans = []
with torch.no_grad():
    for batch_q,batch_t,batch_label,batch_lgb in test_data_generator:
        var_q = torch.from_numpy(batch_q).long()
        var_t = torch.from_numpy(batch_t).long()
        var_lgb = torch.from_numpy(batch_lgb).float()
        var_label = torch.from_numpy(batch_label)
        predict = test_model(var_q,var_t,var_lgb)
        predict = F.softmax(predict,dim=1).detach().cpu()
        ans.extend(predict[:,1].data.tolist())
        y_true.extend(batch_label)

        
ans = np.array(ans)        
print(len(ans))
print(ans[:10])
print(len(y_true))
print(y_true[:10])

from sklearn.metrics import roc_auc_score
# ############################################### 存在Nan值
# for i in ~np.isfinite(ans):
#     if i:
#         print(i)

ans[~np.isfinite(ans)] = 0
print(ans.shape)
roc_auc_score(y_true, ans)

7299
[0.32754898 0.22393286 0.2760759  0.06194067 0.07494764 0.06870329
 0.05847311 0.35875732 0.14952111 0.11457729]
7299
[1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
(7299,)


0.6048066850934366

In [25]:
print(cal_loss_list[-100:])
print(np.average(cal_loss_list[-20:]))

[0.9779124855995178, 1.1770848035812378, 0.906995415687561, 0.9728888869285583, 0.9742032885551453, 0.7176792621612549, 0.7484917044639587, 1.0238046646118164, 0.9729426503181458, 0.8356476426124573, 0.7300366759300232, 0.714935839176178, 0.9595131874084473, 1.21273672580719, 0.8908329010009766, 0.684857189655304, 0.7478893995285034, 0.7168618440628052, 0.6446659564971924, 0.774322509765625, 0.6807888746261597, 0.7696434855461121, 0.7226775884628296, 0.7235695719718933, 0.7567645907402039, 0.7298312187194824, 0.8541026711463928, 0.7591984868049622, 0.8579199314117432, 0.776045560836792, 0.664573609828949, 0.6480347514152527, 0.6838914752006531, 0.6908853054046631, 0.6446559429168701, 0.6452126502990723, 0.7239602208137512, 0.5845644474029541, 0.8050169348716736, 0.7510603070259094, 0.6538920402526855, 0.6918269991874695, 0.6779031157493591, 0.6566629409790039, 0.5742937326431274, 0.6118842959403992, 0.6476324796676636, 0.6596810817718506, 0.6173651218414307, 0.6635706424713135, 0.56511